In [1]:
from abc import ABC, abstractmethod
from dataclasses import dataclass
import os
import openai
import nltk

In [ ]:
openai.api_key = os.environ["OPENAI_API_KEY"]

import requests

messages = [
    {"role" : "system" , "content" : "Kamu adalah asisten yang sangat pemarah dan suka ngomel-ngomel dan berbicara sangat singkat, berbahasa indonesia, tidak suka basa basi, dan cenderung menggunakan kata-kata kasar"}
]


def ask_gpt(prompt):
    global messages
    messages.append(
        {"role" : "user", "content" : prompt}
    )
    response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=messages,
#      max_tokens=1024,
#      n=1,
#      stop=None,
#      temperature=0.7,
    )

    message = response.choices[0].message.content
    messages.append({"role" : "assistant", "content" : message})
    return message


In [ ]:
print(ask_gpt("sebutkan kepada saya apa kira-kira pertanyaan yang penting untuk kita bahas?"))

In [2]:
CONFIG = {}
CONFIG['logdir'] = './log/'

In [2]:
from conversations import Conversation

In [ ]:
del Conversation

In [3]:
all_conversation = {}

def process(from_number, to_number, message, **kwargs):
    """Buat action untuk object Conversation -> buat object apabila belum ada"""
    if from_number not in all_conversation:
        all_conversation.update({from_number : Conversation(from_number, to_number, message)})
    if 'action' in kwargs:
        return all_conversation[from_number].process(kwargs['action'], message)
    


In [4]:
def add_convo(from_number, to_number, message):
    all_conversation.update({from_number : Conversation(from_number, to_number, message)})
    
add_convo("ARDIO", "RONI", "ini pesanku")
add_convo("USER2", "RAPLA", "pesan mati lampu")
add_convo("NUMBER", "3993894", "ini pesanku brother")    

In [40]:
def add(obj, message):
    return (f"my number is {message}")

In [41]:
add_convo("62895352277562@c.us", "6285775300227@c.us", "ini pesanku")

def mama(obj, obj2, obj3):
    return obj2.process(add, obj2.messages,"hello")

In [42]:
mama(add,all_conversation["62895352277562@c.us"],add)

'my number is hello'

In [ ]:
def save_log(from_number):
    """
    cari object di all_conversation dict lalu save jadi log file
    
    """
    if from_number not in all_conversation:
        return print(f"Conversation dengan {from_number} tidak ada")
    dir = CONFIG['logdir']
    filepath = str(from_number) + ".log"
    
        
    # Create the log directory if it doesn't exist
    if not os.path.exists(dir):
        os.makedirs(dir)

    # Write to the log file
    try:
        with open(dir + filepath, "a") as f:
            for i in all_conversation[from_number].messages:
                f.writelines(str(i)+'\n',)
    except Exception as e:
        print("Error writing to file:", str(e))

save_log('ARDIO')

In [ ]:
save_log('ARDIO')

In [ ]:
process("ARDIO", "RONI", "Kamu sedang apa ya?", action=ask_gpt)

In [ ]:
add_convo("62895352277562@c.us", "6285775300227@c.us", "ini pesanku")

In [ ]:
all_conversation['62895352277562@c.us'].send_to_phone("Hello brother")

In [ ]:
all_conversation

In [ ]:
all_conversation['ARDIO'].messages
all_conversation['ARDIO'].add_role_user("Apa kabar kamu dik?")
all_conversation['ARDIO'].add_role_assistant("Lumayan baik lah, apa sih?")
all_conversation['ARDIO'].add_role_user("sholat zhuhur dulu")
all_conversation['ARDIO'].add_role_assistant("oke deh abiku yang ganteng")

In [ ]:
   
if count_words(messages) > 20:
    messages = reduce_conversation(messages)


In [ ]:
30 > 20

In [ ]:
all_conversation['ARDIO'].messages

In [ ]:
from enum import Enum, auto
class Role(Enum):
    SYSTEM = auto()
    USER = auto()
    ASSISTANT = auto()
        
def mars(msg, role: Role):
    roles = {
        "SYSTEM" : 'hey kamu',
        "USER" : 'hey user',
        "ASSISTANT" : "hey assistant"
    }
    print(roles[role.name])

In [ ]:
mars("Hello", Role.SYSTEM)

## BACA LOGS

In [ ]:
import requests
setphone = "62895352277562@c.us"
print(requests.get(f"http://127.0.0.1:8998/print_messages/{setphone}").json()['messages'])
#print(f"http://127.0.0.1:8998/print_messages/{setphone}")

## SET PERSONA

In [ ]:
## send to api whatsapp-web.js open port
from enum import Enum, auto
import requests
from conversations import Role, Message, InjectedMessage, MessageContent, Conversation


def set_message(user_number, message, role: Role):

    url = f"http://127.0.0.1:8998/set_content" # Replace with your endpoint URL
    message = {
        "user_number": user_number, # Replace with the sender number
        "bot_number": "6285775300227@c.us", # Replace with the recipient number in WhatsApp format
        "message": message,
        "role": role.name,
    }

    response = requests.post(url, json=message)

    if response.status_code == 200:
        print("Message sent successfully!")
        print(response.text)
    else:
        print(f"Error sending message. Status code: {response.status_code}")
        print(response.text)

        
# 62895352277562
# 62895391400811 tom
#6287760368781 milt
setphone = "62895352277562@c.us"
set_message(setphone, "Kamu adalah Sauron, kamu berbahasa indonesia ", role=Role.SYSTEM)
set_message(setphone, """
Saya ingin Anda bersikap seperti Lord Sauron dari Seri Lord of the Rings. 
Saya ingin Anda merespons dan menjawab seperti Sauron dengan menggunakan nada, cara, 
dan kosakata yang digunakan Sauron. Jangan menulis penjelasan apa pun. Jawab saja seperti Sauron. 
Anda harus mengetahui semua pengetahuan tentang Sauron. 
""", role=Role.USER)
set_message(setphone, "Ha ha ha aku Sauron", role=Role.ASSISTANT)


## CUSTOM GPT

In [ ]:
from dataclasses import dataclass
from string import Template

@dataclass
class Sys_Message:
    role_system: str = "kamu adalah asisten yang ramah dan membantu."
    role_user: str = "jawab dengan singkat"
    role_assistant: str = "Hallo, ada yang bisa saya bantu?"


def custom_gpt(sys_msg: Sys_Message, prompt: str) -> str:
    
    messages = []
    messages.append({'role' : 'system', 'content' : sys_msg.role_system})
    messages.append({'role' : 'user', 'content' : sys_msg.role_user})
    messages.append({'role' : 'assistant' , 'content' : sys_msg.role_assistant})   
    messages.append({'role': 'user', 'content' : prompt})
    
    response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=messages,
#      max_tokens=1024,
#      n=1,
#      stop=None,
      temperature=0.3,
    )

    message = response.choices[0].message.content
    messages.append({"role" : "assistant", "content" : message})
    #print(messages)
    return message

instruction = Sys_Message(
    """kamu adalah assisten yang baik
        kamu akan menjawab selalu dalam format JSON,
    """,
    """
    Kamu adalah asisten yang pandai dan bisa memformat hasil akhir.
    """    
    ,
    "Hallo, nama saya Maya",
)

prompt = Template("""
    Jawab secara singkat, \n 
    
    $pertanyaan 
    
    buat jawaban dalam bentuk JSON seperti contoh ini:
    
    {
    'pertanyaan' : '..',
    'jawaban' : '..'
    }
    
    tampilkan hanya JSON saja.
    """)

pertanyaan = {'pertanyaan' : "Siapa tokoh paling jahat di Lord of the Rings"}
print(custom_gpt(instruction, prompt.substitute(pertanyaan)))

In [ ]:
print(getattr(prompta.template, 'substitute')(prompta.user_data))

In [ ]:
print(prompt.template.substitute(prompt.user_data))

In [ ]:
aa = Template('hello')

In [ ]:
msg = "Gpt apa kabarnya kamu dik"
msg[0:3].lower()

In [ ]:
#!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import asyncio


class Timer:
    def __init__(self, timeout, callback):
        self._timeout = timeout
        self._callback = callback
        self._task = asyncio.ensure_future(self._job())

    async def _job(self):
        await asyncio.sleep(self._timeout)
        await self._callback()

    def cancel(self):
        self._task.cancel()


async def timeout_callback():
    await asyncio.sleep(0.1)
    print('echo!')


async def main():
    print('\nfirst example:')
    timer = Timer(2, timeout_callback)  # set timer for two seconds
    await asyncio.sleep(2.5)  # wait to see timer works

    print('\nsecond example:')
    timer = Timer(10, timeout_callback)  # set timer for two seconds
    await asyncio.sleep(1)
    #timer.cancel()  # cancel it
    await asyncio.sleep(1.5)  # and wait to see it won't call callback


loop = asyncio.new_event_loop()
asyncio.set_event_loop(loop)
try:
    loop.run_until_complete(main())
finally:
    loop.run_until_complete(loop.shutdown_asyncgens())
    loop.close()

In [ ]:
del loop

In [ ]:
text = 'Mengkonversi karakter 🐍 unicode ke ASCII'
text = text.encode('ascii', errors='ignore').decode()
print(text)


## PUT BOTQUESTION

In [ ]:
# Menyiapkan beberapa data
from conversations import BotQuestion, ConvMode
import json
import requests

botquestion = [BotQuestion(1,"apa sih itu?"), BotQuestion(2,"indi dia")]
user_number = "62895352277562@c.us"

# Mengirim data melalui REST API
url = f'http://localhost:8998/botquestion/{user_number}'
headers = {'Content-type': 'application/json'}
data = json.dumps([{'id': b.id, 
                    'question': b.question, 
                    'answer': b.answer, 
                    'multiplier': b.multiplier, 
                    'score': b.score} for b in botquestion])
print(data)
response = requests.put(url, headers=headers, data=data)

# Menampilkan respons dari server
if response.status_code == 200:
    print('Data berhasil dikirim!')
else:
    print('Terjadi kesalahan saat mengirim data.')


In [ ]:
del BotQuestion

## DISPLAY QUESTIONS

In [ ]:
print(requests.get(f'http://localhost:8998/botq/{user_number}').json())

## SET CONVERSATION MODE

In [ ]:
convmode = ConvMode.ASK
print(requests.get(f'http://localhost:8998/convmode/{user_number}/{convmode}'))

## GET MODE

In [ ]:
print(requests.get(f'http://localhost:8998/getmode/{user_number}').json())

In [ ]:
import nltk

In [ ]:
nltk.word_tokenize(aa)

In [ ]:
Kamu adalah pewawancara pencari kerja yang sangat teliti. 
Beri skor dari koherensi antara pertanyaan dan jawaban dari angka 1 sd 5.

Pertanyaan: Siapakah namamu?
Jawaban: Nama saya Irza
    
Kamu akan menjawab dalam format JSON dalam bentuk seperti contoh ini:
    
{
    'pertanyaan' : '..',
    'jawaban' : '..',
    'koherensi' : '..',
}

Tampilkan hanya JSON saja.

In [ ]:
Jawab secara singkat, \n 
    
    Siapakah tokoh paling jahat di Harry Potter
    
    buat jawaban dalam bentuk JSON seperti contoh ini:
    
    {
    'pertanyaan' : '..',
    'jawaban' : '..'
    }
    
    tampilkan hanya JSON saja.

In [ ]:
import asyncio

class Timer:
    def __init__(self, wait_time):
        self.wait_time = wait_time

    async def start_timer(self):
        await asyncio.sleep(self.wait_time)
        print(f"{self.wait_time} seconds have passed")

async def main():
    timers = [Timer(10), Timer(6), Timer(19)]
    await asyncio.gather(*[timer.start_timer() for timer in timers])

asyncio.run(main())
